In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
!pip install -q git+https://github.com/huggingface/transformers.git
!pip install -q git+https://github.com/huggingface/datasets.git "dill<0.3.5" seqeval
!pip install datasets transformers==4.28.0

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 34.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 52.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.9/86.9 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13

In [ ]:
# Pull the preprocess.py script to process the ZIP file exported from UBIAI
!rm -r layoutlmv3FineTuning
!git clone -b master https://github.com/UBIAI/layoutlmv3FineTuning.git

rm: cannot remove 'layoutlmv3FineTuning': No such file or directory
Cloning into 'layoutlmv3FineTuning'...
remote: Enumerating objects: 40, done.
remote: Counting objects: 100% (40/40), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 40 (delta 14), reused 40 (delta 14), pack-reused 0
Receiving objects: 100% (40/40), 12.36 KiB | 2.06 MiB/s, done.
Resolving deltas: 100% (14/14), done.


In [ ]:
#copy the ubiai anoteted data from drive
IOB_DATA_PATH = "/content/drive/MyDrive/UBIAI/test_anotation_sep_7.zip"


# Copy and extract the dataset
!cd /content/
!rm -r data
!mkdir data
!cp "$IOB_DATA_PATH" data/dataset.zip
!cd data && unzip -q dataset
!cd ..


rm: cannot remove 'data': No such file or directory


**convert datset into layoutmlv3 format**

In [ ]:
# Set preprocessing arguments
TEST_SIZE=0.1
DATA_OUTPUT_PATH="/content/"

import pandas as pd
import numpy as np
import os
import argparse
from datasets.features import ClassLabel
from transformers import AutoProcessor
from sklearn.model_selection import train_test_split
from datasets import Features, Sequence, ClassLabel, Value, Array2D, Array3D, Dataset
from datasets import Image as Img
from PIL import Image

import warnings
warnings.filterwarnings('ignore')


def read_text_file(file_path):
    print(file_path)
    with open(file_path, 'r') as f:
        return (f.readlines())


def prepare_examples(examples):
  images = examples[image_column_name]
  words = examples[text_column_name]
  boxes = examples[boxes_column_name]
  word_labels = examples[label_column_name]

  encoding = processor(images, words, boxes=boxes, word_labels=word_labels,
                       truncation=True, padding="max_length")

  return encoding

def get_zip_dir_name():
     return '/content/data'

def filter_out_unannotated(example):
    tags = example['ner_tags']
    return not all([tag == label2id['O'] for tag in tags])



if __name__ == '__main__':

    # parser = argparse.ArgumentParser()
    # parser.add_argument('--valid_size')
    # parser.add_argument('--output_path')
    # args = parser.parse_args()

    TEST_SIZE = TEST_SIZE
    OUTPUT_PATH = DATA_OUTPUT_PATH

    os.makedirs(OUTPUT_PATH, exist_ok=True)
    files = {}
    zip_dir_name = get_zip_dir_name()
    if zip_dir_name:
        print()
        files['train_box'] = read_text_file(os.path.join(
            os.curdir, 'data', f'{zip_dir_name}//9e957911-0ca4-49df-8ca2-194315772d65_box.txt'))
        files['train_image'] = read_text_file(os.path.join(
            os.curdir, 'data', f'{zip_dir_name}//9e957911-0ca4-49df-8ca2-194315772d65_image.txt'))
        files['train'] = read_text_file(os.path.join(
            os.curdir, 'data', f'{zip_dir_name}//9e957911-0ca4-49df-8ca2-194315772d65.txt'))
    else:
        for f in os.listdir():
            if f.endswith('.txt') and f.find('box') != -1:
                files['train_box'] = read_text_file(os.path.join(os.curdir, f))
            elif f.endswith('.txt') and f.find('image') != -1:
                files['train_image'] = read_text_file(
                    os.path.join(os.curdir, f))
            elif f.endswith('.txt') and f.find('labels') == -1:
                files['train'] = read_text_file(os.path.join(os.curdir, f))

    assert(len(files['train']) == len(files['train_box']))
    assert(len(files['train_box']) == len(files['train_image']))
    assert(len(files['train_image']) == len(files['train']))

    images = {}
    for i, row in enumerate(files['train_image']):# image = Image.open(dataset['train'][10]['image'])
# image = image.convert("RGB")
        if row != '\n':
            image_name = row.split('\t')[-1]
            images.setdefault(image_name.replace('\n', ''), []).append(i)

    words, bboxes, ner_tags, image_path = [], [], [], []
    for image, rows in images.items():
        words.append([row.split('\t')[0].replace('\n', '')
                     for row in files['train'][rows[0]:rows[-1]+1]])
        ner_tags.append([row.split('\t')[1].replace('\n', '')
                        for row in files['train'][rows[0]:rows[-1]+1]])
        bboxes.append([box.split('\t')[1].replace('\n', '')
                      for box in files['train_box'][rows[0]:rows[-1]+1]])
        if zip_dir_name:
            image_path.append(f"/content/data/{image}")
        else:
            image_path.append(f"/content/data/{image}")

    labels = list(set([tag for doc_tag in ner_tags for tag in doc_tag]))
    id2label = {v: k for v, k in enumerate(labels)}
    label2id = {k: v for v, k in enumerate(labels)}

    dataset_dict = {
        'id': range(len(words)),
        'tokens': words,
        'bboxes': [[list(map(int, bbox.split())) for bbox in doc] for doc in bboxes],
        'ner_tags': [[label2id[tag] for tag in ner_tag] for ner_tag in ner_tags],
        'image': [Image.open(path).convert("RGB") for path in image_path]
    }

    #raw features
    features = Features({
        'id': Value(dtype='string', id=None),
        'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
        'bboxes': Sequence(feature=Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None), length=-1, id=None),
        'ner_tags': Sequence(feature=ClassLabel(num_classes=len(labels), names=labels, names_file=None, id=None), length=-1, id=None),
        'image': Img(decode=True, id=None)
    })

    full_data_set = Dataset.from_dict(dataset_dict, features=features)
    dataset = full_data_set.train_test_split(test_size=TEST_SIZE)
    dataset["train"] = dataset["train"].filter(filter_out_unannotated)
    processor = AutoProcessor.from_pretrained(
        "microsoft/layoutlmv3-base", apply_ocr=False)

    features = dataset["train"].features
    column_names = dataset["train"].column_names
    image_column_name = "image"
    text_column_name = "tokens"
    boxes_column_name = "bboxes"
    label_column_name = "ner_tags"
    # we need to define custom features for `set_format` (used later on) to work properly
    features = Features({
        'pixel_values': Array3D(dtype="float32", shape=(3, 224, 224)),
        'input_ids': Sequence(feature=Value(dtype='int64')),
        'attention_mask': Sequence(Value(dtype='int64')),
        'bbox': Array2D(dtype="int64", shape=(512, 4)),
        'labels': Sequence(ClassLabel(names=labels)),
    })

    train_dataset = dataset["train"].map(
        prepare_examples,
        batched=True,
        remove_columns=column_names,
        features=features,
    )
    eval_dataset = dataset["test"].map(
        prepare_examples,
        batched=True,
        remove_columns=column_names,
        features=features,
    )
    train_dataset.set_format("torch")
    if not OUTPUT_PATH.endswith('/'):
        OUTPUT_PATH += '/'
    train_dataset.save_to_disk(f'{OUTPUT_PATH}train_split')
    eval_dataset.save_to_disk(f'{OUTPUT_PATH}eval_split')
    dataset.save_to_disk(f'{OUTPUT_PATH}raw_data')


/content/data//9e957911-0ca4-49df-8ca2-194315772d65_box.txt
/content/data//9e957911-0ca4-49df-8ca2-194315772d65_image.txt
/content/data//9e957911-0ca4-49df-8ca2-194315772d65.txt


Filter:   0%|          | 0/24 [00:00<?, ? examples/s]

Map:   0%|          | 0/24 [00:00<?, ? examples/s]

Map:   0%|          | 0/3 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/24 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/3 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/24 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/3 [00:00<?, ? examples/s]

In [ ]:
# Load the dataset post-process
from datasets import load_metric
from transformers import TrainingArguments, Trainer
from transformers import LayoutLMv3ForTokenClassification, AutoProcessor
from transformers.data.data_collator import default_data_collator
import torch

lood dataset


In [ ]:
from datasets import load_from_disk
train_dataset = load_from_disk(f'/content/train_split')
row_dataset = load_from_disk(f'/content/raw_data')

print(row_dataset)
eval_dataset = train_dataset




DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'bboxes', 'ner_tags', 'image'],
        num_rows: 24
    })
    test: Dataset({
        features: ['id', 'tokens', 'bboxes', 'ner_tags', 'image'],
        num_rows: 3
    })
})


In [ ]:
import shutil

folder_path_on_colab = '/content/raw_data'  # Replace with the actual folder path
zip_file_path_on_colab = '/content/raw_data_finetune_layoumlv3_sep_7'  # Replace with the desired zip file path

shutil.make_archive(zip_file_path_on_colab, 'zip', folder_path_on_colab)


'/content/raw_data_finetune_layoumlv3_sep_7.zip'

In [ ]:
from google.colab import files

files.download('/content/raw_data_finetune_layoumlv3.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
label_list = train_dataset.features["labels"].feature.names
num_labels = len(label_list)
label2id, id2label = dict(), dict()

In [ ]:
num_labels = len(label_list)
label2id, id2label = dict(), dict()
for i, label in enumerate(label_list):
    label2id[label] = i
    id2label[i] = label

In [ ]:
import numpy as np
from datasets import load_metric

metric = load_metric("seqeval")
return_entity_level_metrics = False

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels, zero_division='warn')


    if return_entity_level_metrics:
        # Unpack nested dictionaries
        final_results = {}
        for key, value in results.items():
            if isinstance(value, dict):
                for n, v in value.items():
                    final_results[f"{key}_{n}"] = v
            else:
                final_results[key] = value
        return final_results
    else:
        return {
            "precision": results["overall_precision"],
            "recall": results["overall_recall"],
            "f1": results["overall_f1"],
            "accuracy": results["overall_accuracy"],
        }


In [ ]:
from transformers import LayoutLMv3ForTokenClassification, AutoProcessor, TrainingArguments, Trainer
from transformers.data.data_collator import default_data_collator

In [ ]:
model = LayoutLMv3ForTokenClassification.from_pretrained("microsoft/layoutlmv3-base",
                                                         id2label=id2label,
                                                         label2id=label2id)
processor = AutoProcessor.from_pretrained("microsoft/layoutlmv3-base", apply_ocr=False)


Some weights of LayoutLMv3ForTokenClassification were not initialized from the model checkpoint at microsoft/layoutlmv3-base and are newly initialized: ['classifier.dense.bias', 'classifier.out_proj.bias', 'classifier.dense.weight', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
NUM_TRAIN_EPOCHS = 50
PER_DEVICE_TRAIN_BATCH_SIZE = 1
PER_DEVICE_EVAL_BATCH_SIZE = 1
LEARNING_RATE = 4e-5

In [ ]:
training_args = TrainingArguments(output_dir="layoutmlV3FineTuneModel",
                                  num_train_epochs=NUM_TRAIN_EPOCHS,
                                  logging_strategy="epoch",
                                  save_total_limit=1,
                                  per_device_train_batch_size=PER_DEVICE_TRAIN_BATCH_SIZE,
                                  per_device_eval_batch_size=PER_DEVICE_EVAL_BATCH_SIZE,
                                  learning_rate=LEARNING_RATE,
                                  evaluation_strategy="epoch",
                                  save_strategy="epoch",
                                  load_best_model_at_end=True,
                                  metric_for_best_model="f1")

In [ ]:
# Initialize our Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=processor,
    data_collator=default_data_collator,
    compute_metrics=compute_metrics,
)

# Train the model
trainer.train()

# Evaluate the model
evaluation_results = trainer.evaluate()

# Display evaluation results
print("Evaluation Results:")
print(evaluation_results)

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,1.716500,1.401200,0.000000,0.000000,0.000000,0.704279
2,1.294100,0.975483,0.140625,0.142105,0.141361,0.780411
3,1.114600,0.810279,0.259146,0.447368,0.328185,0.811661
4,0.822000,1.048943,0.512915,0.731579,0.603037,0.814807
5,0.874700,0.530092,0.575342,0.663158,0.616137,0.906879
6,0.746200,0.622023,0.707317,0.763158,0.734177,0.876049
7,0.569400,0.401480,0.521236,0.710526,0.601336,0.931837
8,0.369600,0.253818,0.728972,0.821053,0.772277,0.969799
9,0.281000,0.233674,0.740566,0.826316,0.781095,0.970428
10,0.299100,0.256854,0.740088,0.884211,0.805755,0.972315


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,1.716500,1.401200,0.000000,0.000000,0.000000,0.704279
2,1.294100,0.975483,0.140625,0.142105,0.141361,0.780411
3,1.114600,0.810279,0.259146,0.447368,0.328185,0.811661
4,0.822000,1.048943,0.512915,0.731579,0.603037,0.814807
5,0.874700,0.530092,0.575342,0.663158,0.616137,0.906879
6,0.746200,0.622023,0.707317,0.763158,0.734177,0.876049
7,0.569400,0.401480,0.521236,0.710526,0.601336,0.931837
8,0.369600,0.253818,0.728972,0.821053,0.772277,0.969799
9,0.281000,0.233674,0.740566,0.826316,0.781095,0.970428
10,0.299100,0.256854,0.740088,0.884211,0.805755,0.972315


Evaluation Results:
{'eval_loss': 0.005510688293725252, 'eval_precision': 1.0, 'eval_recall': 1.0, 'eval_f1': 1.0, 'eval_accuracy': 0.9997902684563759, 'eval_runtime': 70.2785, 'eval_samples_per_second': 0.285, 'eval_steps_per_second': 0.285, 'epoch': 50.0}


In [ ]:
print(evaluation_results)

{'eval_loss': 0.005510688293725252, 'eval_precision': 1.0, 'eval_recall': 1.0, 'eval_f1': 1.0, 'eval_accuracy': 0.9997902684563759, 'eval_runtime': 70.2785, 'eval_samples_per_second': 0.285, 'eval_steps_per_second': 0.285, 'epoch': 50.0}


In [ ]:
!pip install huggingface_hub

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
print(evaluation_results)

{'eval_loss': 0.005510688293725252, 'eval_precision': 1.0, 'eval_recall': 1.0, 'eval_f1': 1.0, 'eval_accuracy': 0.9997902684563759, 'eval_runtime': 70.2785, 'eval_samples_per_second': 0.285, 'eval_steps_per_second': 0.285, 'epoch': 50.0}


In [ ]:
trainer.model.push_to_hub("dwitidibyajyoti/layoutmlv3PdfParsing")

CommitInfo(commit_url='https://huggingface.co/dwitidibyajyoti/layoutmlv3PdfParsing/commit/ac45d2e9dcaf0d5d9ef6b63a7af3b232021d2db3', commit_message='Upload LayoutLMv3ForTokenClassification', commit_description='', oid='ac45d2e9dcaf0d5d9ef6b63a7af3b232021d2db3', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
#check if model have the toknizer or not

from transformers import AutoTokenizer

model_name = "dwitidibyajyoti/layoutmlv3PdfParsing"
try:
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    print("Tokenizer exists for", model_name)
except Exception as e:
    print("Tokenizer not found for", model_name)

Tokenizer not found for dwitidibyajyoti/layoutmlv3PdfParsing


In [ ]:
from transformers import AutoModelForSequenceClassification,LayoutLMv3ForSequenceClassification,LayoutLMv3Tokenizer
import torch

DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"
# model = AutoModelForSequenceClassification.from_pretrained("dwitidibyajyoti/layoutmlv3PdfParsing")
model = LayoutLMv3ForSequenceClassification.from_pretrained(
    "dwitidibyajyoti/layoutmlv3PdfParsing"
)
tokenizer = LayoutLMv3Tokenizer.from_pretrained("microsoft/layoutlmv3-base")
model = model.eval().to(DEVICE)

ModuleNotFoundError: ignored

In [ ]:
!pip install pytesseract

In [ ]:
model.config.id2label

{0: 'S-IGNORE',
 1: 'I-IGNORE',
 2: 'E-TABLE',
 3: 'I-TABLE',
 4: 'E-VALUE',
 5: 'B-VALUE',
 6: 'B-IGNORE',
 7: 'S-VALUE',
 8: 'I-VALUE',
 9: 'E-IGNORE',
 10: 'B-TABLE',
 11: 'S-KEY',
 12: 'O'}

In [ ]:
import pytesseract

def perform_ocr(image):
    # Perform OCR on the image using pytesseract
    ocr_results = pytesseract.image_to_data(image, output_type=pytesseract.Output.DICT)

    ocr_data = []
    for i in range(len(ocr_results["text"])):
        word = ocr_results["text"][i]
        bounding_box = {
            "left": ocr_results["left"][i],
            "top": ocr_results["top"][i],
            "width": ocr_results["width"][i],
            "height": ocr_results["height"][i]
        }
        ocr_data.append({"word": word, "bounding_box": bounding_box})

    return ocr_data

In [ ]:
from typing import List
def scale_bounding_box(box, width_scale : float = 1.0, height_scale : float = 1.0) -> List[int]:
    return [
        int(box['left'] * width_scale),
        int(box['top'] * height_scale),
        int(box['width'] * width_scale),
        int(box['height'] * height_scale)
    ]


In [ ]:
import torch

# def predict_document_image(
#     image_path,
#     model,
#     processor):

#     json_path = image_path.with_suffix(".json")
#     with json_path.open("r") as f:
#         ocr_result = json.load(f)

#         with Image.open(image_path).convert("RGB") as image:

#             width, height = image.size
#             width_scale = 1000 / width
#             height_scale = 1000 / height

#             words = []
#             boxes = []
#             for row in ocr_result:
#                 boxes.append(
#                     scale_bounding_box(
#                         row["bounding_box"],
#                         width_scale,
#                         height_scale
#                     )
#                 )
#                 words.append(row["word"])

#             encoding = processor(
#                 image,
#                 words,
#                 boxes=boxes,
#                 max_length=512,
#                 padding="max_length",
#                 truncation=True,
#                 return_tensors="pt"
#             )

#     with torch.inference_mode():
#         output = model(
#             input_ids=encoding["input_ids"].to(DEVICE),
#             attention_mask=encoding["attention_mask"].to(DEVICE),
#             bbox=encoding["bbox"].to(DEVICE),
#             pixel_values=encoding["pixel_values"].to(DEVICE)id2label
#         )

#     predicted_class = output.logits.argmax()
#     return model.config.id2label[predicted_class.item()]


def predict_document_image(image_path, model, processor):
    # Load the image
    with Image.open(image_path).convert("RGB") as image:
        width, height = image.size
        width_scale = 1000 / width
        height_scale = 1000 / height

        # Perform OCR on the image
        ocr_result = perform_ocr(image)  # Replace with your OCR implementation
        print(ocr_result[0]['bounding_box'])
        words = []
        boxes = []
        for row in ocr_result:
            boxes.append(
                scale_bounding_box(
                    row["bounding_box"],
                    width_scale,
                    height_scale
                )
            )
            words.append(row["word"])

        encoding = processor(
            image,
            words,
            boxes=boxes,
            max_length=512,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )

    with torch.inference_mode():
        output = model(
            input_ids=encoding["input_ids"].to(DEVICE),
            attention_mask=encoding["attention_mask"].to(DEVICE),
            bbox=encoding["bbox"].to(DEVICE),
            pixel_values=encoding["pixel_values"].to(DEVICE)
        )

    predicted_class = output.logits.argmax()
    print(encoding["input_ids"])


    return model.config.id2label[predicted_class.item()]


In [ ]:
from transformers import LayoutLMv3FeatureExtractor ,LayoutLMv3TokenizerFast, LayoutLMv3Processor
feature_extractor = LayoutLMv3FeatureExtractor(apply_ocr=False)
tokenizer = LayoutLMv3TokenizerFast.from_pretrained(
    "microsoft/layoutlmv3-base"
)
processor = LayoutLMv3Processor(feature_extractor, tokenizer)

/usr/local/lib/python3.10/dist-packages/transformers/models/layoutlmv3/feature_extraction_layoutlmv3.py:30: FutureWarning: The class LayoutLMv3FeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use LayoutLMv3ImageProcessor instead.
  warnings.warn(


In [ ]:
# test_images = train_test_split('/content/images', test_size=.2)
!mkdir image

In [ ]:
from tqdm import tqdm
labels = []
predictions = []
# for image_path in tqdm(test_images):
#     labels.append(image_path)

predictions.append(predict_document_image('/content/images/TS SP3 MND MOTOR 5041_June2023 + Annexures_page-0013.jpg', model, processor))

print(predictions)


NameError: ignored